# Binary Search

Implement a binary search algorithm.

Searching a sorted collection is a common task. A dictionary is a sorted
list of word definitions. Given a word, one can find its definition. A
telephone book is a sorted list of people's names, addresses, and
telephone numbers. Knowing someone's name allows one to quickly find
their telephone number and address.

If the list to be searched contains more than a few items (a dozen, say)
a binary search will require far fewer comparisons than a linear search,
but it imposes the requirement that the list be sorted.

In computer science, a binary search or half-interval search algorithm
finds the position of a specified input value (the search "key") within
an array sorted by key value.

In each step, the algorithm compares the search key value with the key
value of the middle element of the array.

If the keys match, then a matching element has been found and the range of indices that equal the search key value are returned.

Otherwise, if the search key is less than the middle element's key, then
the algorithm repeats its action on the sub-array to the left of the
middle element or, if the search key is greater, on the sub-array to the
right.

If the remaining array to be searched is empty, then the key cannot be
found in the array and a special "not found" indication is returned. Search methods in Julia typically return an empty range located at the insertion point in this case.

A binary search halves the number of items to check with each iteration,
so locating an item (or determining its absence) takes logarithmic time.
A binary search is a dichotomic divide and conquer search algorithm.

**For simplification, you can assume that all elements of the list to be searched are unique.** Feel free to implement a solution that works on lists with non-unique elements as a bonus task.

## Bonus task
Implement keyword arguments `by`, `lt` and `rev` so that `by` specifies a transformation applied to all elements of the list, `lt` specifies a comparison and `rev` specifies if the list is ordered in reverse.

## Source

Wikipedia [http://en.wikipedia.org/wiki/Binary_search_algorithm](http://en.wikipedia.org/wiki/Binary_search_algorithm)

Some phrases above and the bonus tasks are taken from the [Julia base documentation (MIT license)](https://docs.julialang.org/en/v1/base/sort/#Base.Sort.searchsorted) of `searchsorted`.


## Version compatibility
This exercise has been tested on Julia versions >=1.0.

## Submitting Incomplete Solutions
It's possible to submit an incomplete solution so you can see how others have completed the exercise.


In [41]:
# submit
using Match

function binarysearch(as, t; rev::Bool=false, by=nothing, lt=isless) # Julia function to do this searchsorted
    isless = (<)
    transform = isnothing(by) ? (as,t) : (map(by, as), by(t))
    search((as,t)) = isempty(as) ? (1,0) : searcher(sort(as,lt=lt), 1, length(as), t)
    flip((t1,t2)) = rev ? (length(as) - t2 + 1, length(as) - t1 + 1) : (t1,t2)
    format_output((t1,t2)) = t1:t2
    
    function searcher(as::Array{S}, l::Int64, r::Int64, t::T) where {S, T <: Number}
        @match (as, l, r, t) begin
            ([], _,_,_) => (1,0)
            (_as, l, r, _t), if l > r end  => (l,r)
            ( as, l, r,  t), if lt(as[div(l + r, 2)],t) end => searcher(as, div(l + r, 2) + 1, r, t)
            ( as, l, r,  t), if !lt(as[div(l + r, 2)],t) && !isequal(as[div(l + r, 2)],t) end => searcher(as, l, div(l + r, 2) - 1 , t)
            (_as, l, r, _t) =>  (div(l + r,2), div(l + r,2))
        end
    end
    
    transform |> search |> flip |> format_output
    
end


binarysearch (generic function with 1 method)

In [43]:
# canonical data version: 1.2.0

using Test

include("binary-search.jl")

@testset "default binary search" begin
    @testset "value in array" begin
        @test binarysearch([6], 6) == 1:1
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 6) == 4:4
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 1) == 1:1
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 11) == 7:7
        @test binarysearch([1, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 634], 144) == 10:10
        @test binarysearch([1, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377], 21) == 6:6
    end
    @testset "value not in array" begin
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 7) == 5:4
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 0) == 1:0
        @test binarysearch([1, 3, 4, 6, 8, 9, 11], 13) == 8:7
        @test binarysearch([], 1) == 1:0
    end
end

@testset "bonus tasks" begin
    @testset "reverse search" begin
        @testset "value in array" begin
            @test binarysearch([6], 6, rev = true) == 1:1
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 6, rev = true) == 4:4
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 1, rev = true) == 7:7
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 11, rev = true) == 1:1
            @test binarysearch([634, 377, 233, 144, 89, 55, 34, 21, 13, 8, 5, 3, 1], 144, rev = true) == 4:4
            @test binarysearch([377, 233, 144, 89, 55, 34, 21, 13, 8, 5, 3, 1], 21, rev = true) == 7:7
        end
        @testset "value not in array" begin
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 7, rev = true) == 4:3
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 0, rev = true) == 8:7
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 13, rev = true) == 1:0
            @test binarysearch([], 1, rev = true) == 1:0
        end
    end

    @testset "apply transformation" begin
        @testset "value in array" begin
            @test binarysearch([5.5], 6, by = round) == 1:1
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 6, by = round) == 4:4
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 1, by = round) == 1:1
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 11, by = round) == 7:7
            @test binarysearch([1, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 634], 144.4, by = round) == 10:10
            @test binarysearch([1, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377], 20.6, by = round) == 6:6
        end
        @testset "value not in array" begin
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 7, by = round) == 5:4
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 0, by = round) == 1:0
            @test binarysearch([1.1, 2.9, 4.4, 5.5, 8.1, 9.0, 10.8], 13, by = round) == 8:7
            @test binarysearch([], 1, by = round) == 1:0
        end
    end

    @testset "compare with > instead of <" begin
        # this is equivalent to searching in reverse order
        @testset "value in array" begin
            @test binarysearch([6], 6, lt = >) == 1:1
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 6, lt = >) == 4:4
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 1, lt = >) == 7:7
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 11, lt = >) == 1:1
            @test binarysearch([634, 377, 233, 144, 89, 55, 34, 21, 13, 8, 5, 3, 1], 144, lt = >) == 4:4
            @test binarysearch([377, 233, 144, 89, 55, 34, 21, 13, 8, 5, 3, 1], 21, lt = >) == 7:7
        end
        @testset "value not in array" begin
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 7, lt = >) == 4:3
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 0, lt = >) == 8:7
            @test binarysearch([11, 9, 8, 6, 4, 3, 1], 13, lt = >) == 1:0
            @test binarysearch([], 1, lt = >) == 1:0
        end
    end
end


Test Summary:         | Pass  Total
default binary search |   10     10
Test Summary: | Pass  Total
bonus tasks   |   30     30


Test.DefaultTestSet("bonus tasks", Any[DefaultTestSet("reverse search", Any[DefaultTestSet("value in array", Any[], 6, false), DefaultTestSet("value not in array", Any[], 4, false)], 0, false), DefaultTestSet("apply transformation", Any[DefaultTestSet("value in array", Any[], 6, false), DefaultTestSet("value not in array", Any[], 4, false)], 0, false), DefaultTestSet("compare with > instead of <", Any[DefaultTestSet("value in array", Any[], 6, false), DefaultTestSet("value not in array", Any[], 4, false)], 0, false)], 0, false)

In [ ]:
# To submit your exercise, you need to save your solution in a file called binary-search.jl before using the CLI.
# You can either create it manually or use the following functions, which will automatically
# save every notebook cell starting with `# submit` in that file.

# Pkg.add("Exercism")
# using Exercism
# Exercism.create_submission("binary-search")


In [55]:
binarysearch([6.0], 5.6)

1:0

In [7]:

?sort!
         

search: sort! partialsort! sortperm! partialsortperm! setproperty! sort sortperm



```
sort!(v; alg::Algorithm=defalg(v), lt=isless, by=identity, rev::Bool=false, order::Ordering=Forward)
```

Sort the vector `v` in place. `QuickSort` is used by default for numeric arrays while `MergeSort` is used for other arrays. You can specify an algorithm to use via the `alg` keyword (see Sorting Algorithms for available algorithms). The `by` keyword lets you provide a function that will be applied to each element before comparison; the `lt` keyword allows providing a custom "less than" function; use `rev=true` to reverse the sorting order. These options are independent and can be used together in all possible combinations: if both `by` and `lt` are specified, the `lt` function is applied to the result of the `by` function; `rev=true` reverses whatever ordering specified via the `by` and `lt` keywords.

# Examples

```jldoctest
julia> v = [3, 1, 2]; sort!(v); v
3-element Array{Int64,1}:
 1
 2
 3

julia> v = [3, 1, 2]; sort!(v, rev = true); v
3-element Array{Int64,1}:
 3
 2
 1

julia> v = [(1, "c"), (3, "a"), (2, "b")]; sort!(v, by = x -> x[1]); v
3-element Array{Tuple{Int64,String},1}:
 (1, "c")
 (2, "b")
 (3, "a")

julia> v = [(1, "c"), (3, "a"), (2, "b")]; sort!(v, by = x -> x[2]); v
3-element Array{Tuple{Int64,String},1}:
 (3, "a")
 (2, "b")
 (1, "c")
```

---

```
sort!(A; dims::Integer, alg::Algorithm=defalg(v), lt=isless, by=identity, rev::Bool=false, order::Ordering=Forward)
```

Sort the multidimensional array `A` along dimension `dims`. See [`sort!`](@ref) for a description of possible keyword arguments.

To sort slices of an array, refer to [`sortslices`](@ref).

!!! compat "Julia 1.1"
    This function requires at least Julia 1.1.


# Examples

```jldoctest
julia> A = [4 3; 1 2]
2×2 Array{Int64,2}:
 4  3
 1  2

julia> sort!(A, dims = 1); A
2×2 Array{Int64,2}:
 1  2
 4  3

julia> sort!(A, dims = 2); A
2×2 Array{Int64,2}:
 1  2
 3  4
```


In [47]:
map(identity,a) 

7-element Array{Int64,1}:
  1
  3
  4
  6
  8
  9
 11